In [ ]:
import asyncio
import logging
import sys

from astropy.time import Time

from lsst.ts import salobj

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
d = salobj.Domain()
CCCamera = salobj.Remote(d, 'CCCamera')
await CCCamera.start_task

In [ ]:
date = Time.now()
group_id = f'CALSET_{date.tai.strftime("%Y%m%d_%H%M")}'
images = 3
wait_period = 20
for i in range(images):
    print(f"Image {i + 1}")
    try:
        CCCamera.cmd_takeImages.set(numImages=1,
                                    expTime=0.,
                                    shutter=False,
                                    imageType="BIAS",
                                    groupId=group_id,
                                    science=True,
                                    guide=False,
                                    wfs=False)
        ack = await CCCamera.cmd_takeImages.start(timeout=60)
    except salobj.AckError as ae:
        ack = ae.ackcmd
    print(f"ack={ack.ack}, ack.error={ack.error}, ackcmd.result={ack.result}")
    if i + 1 < images:
        await asyncio.sleep(wait_period)